In [2]:
import numpy as np
import pandas as pd
import keras as ks
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

Using TensorFlow backend.


In [3]:
##Reference Code
# louis = sp.search(q='Louis the Child', type='artist')
# louis = louis['artists']['items'][0]
# RL = sp.search(q='RL Grime', type='artist')
# RL = RL['artists']['items'][0]
# print(louis['genres'])
# print(RL['genres'])

# artist = sp.search(q='Louis the Child', limit=50, market='US', offset=0, type='track')

# tracks = artist['tracks']
# items = tracks['items']
# ids = [item['id'] for item in items]
# feats = sp.audio_features(ids)
# print(type(feats[0]))
# track = items[0]['uri']
# analysis = sp.audio_analysis(track)
# features = sp.audio_features(track)
# print(type(analysis), type(features))
# print(analysis.keys())
# print(features)
# for track in items:
#   print(track['name'])

# Using Spotify API Python wrapper to look at data analysis

### Initialize spotipy object with client credentials

In [9]:
tracks = pd.DataFrame()
sp = spotipy.Spotify(client_credentials_manager=\
                     SpotifyClientCredentials(client_id='17e5b5704b914080a62aef78309e09a3', \
                                              client_secret='171d0b70bcd145dabf1d697233541a6d'))

This section will try and pull as much EDM data I can from Spotify to load into our dataframe. 

In [10]:
def show_recommendations_for_artist(artist):
    #returns a list of recommended artists based on a given artist
    artists = []
    results = sp.recommendations(seed_artists = [artist['id']])
    for track in results['tracks']:
        artists.append(track['artists'][0]['name'])
    return artists

In these next two cells I'm creating a sample list of DJs, then using Spotify's recommendation service to generate a much bigger list, as long as the artist is associated with the 'edm' genre.

In [11]:
djs = ['Calvin Harris', 'Diplo', 'Louis the Child', \
       'RL Grime', 'Alison Wonderland', 'Odesza', 'Avicii', \
       'Zedd', 'Tiesto', 'Martin Garrix', 'Skrillex', 'Steve Aoki', \
       'Marshmello', 'Porter Robinson', 'The Chainsmokers', 'Kygo', \
       'Major Lazer', 'DJ Snake']
djset = set(djs)
artists = []
for dj in djs:
        artists.append(sp.search(q=dj, type='artist')['artists']['items'][0])


In [12]:
print(djs)

['Calvin Harris', 'Diplo', 'Louis the Child', 'RL Grime', 'Alison Wonderland', 'Odesza', 'Avicii', 'Zedd', 'Tiesto', 'Martin Garrix', 'Skrillex', 'Steve Aoki', 'Marshmello', 'Porter Robinson', 'The Chainsmokers', 'Kygo', 'Major Lazer', 'DJ Snake']


In [13]:
for a in artists:
        recs = show_recommendations_for_artist(a)
        for rec in recs:
            if rec not in djset and 'edm' in sp.search(q=rec, type='artist')['artists']['items'][0]['genres']:
                djs.append(rec)
                djset.add(rec)

## Now we have a good list of DJs, let's get them into a dataframe

Now, I use Spotify's API to generate a set of features for each song, and storing the feature representation of each song in a 2D list.

In [16]:
data = []
for dj in djs:
    items = sp.search(q=dj, limit=50, offset=0, market='US', type='track')['tracks']['items']
    ids = [item['id'] for item in items]
    if ids:
        ft = [np.array(list(item.values())[:11]) for item in sp.audio_features(ids) if item is not None]
        data += ft

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=None)

Pandas conversion and saving results into a CSV for later use

In [ ]:
npdata = np.array(data)
df = pd.DataFrame(npdata)
df.to_csv('data.csv', index=False)

# Now we need to get a good set of data that represents non-edm songs


I'm going to use the exact same process from above, however this time I'll build a list based on artists that are NOT associated with 'edm'.

In [158]:
nondjs = ['Beyonce', 'The Shins', 'The Who', 'Zac Brown Band', \
           'George Harrison', 'Kendrick Lamar', 'Chance the Rapper', \
           'Miley Cyrus', 'Snoop Dogg', 'Dr. Dre', 'Eminem', 'Bob Marley', \
           'Duke Ellington', 'Miles Davis', 'The Beach Boys', 'The Growlers', 'Five Seconds of Summer']
nondjset = set(nondjs)
artists2 = []
for nondj in nondjs:
    artists2.append(sp.search(q=nondj, type='artist')['artists']['items'][0])

In [ ]:
for a in artists2:
        recs = show_recommendations_for_artist(a)
        for rec in recs:
            try:
                if rec not in nondjset and 'edm' not in sp.search(q=rec, type='artist')['artists']['items'][0]['genres']:
                    nondjs.append(rec)
                    nondjset.add(rec)
            except:
                print('Nothing here')

In [165]:
data2 = []
for nondj in nondjs:
    items = sp.search(q=nondj, limit=50, offset=0, market='US', type='track')['tracks']['items']
    ids = [item['id'] for item in items]
    if ids:
        ft = [np.array(list(item.values())[:11]) for item in sp.audio_features(ids) if item is not None]
        data2 += ft

In [173]:
npdata2 = np.array(data2)
df2 = pd.DataFrame(npdata2)
df2.to_csv('data2.csv', index=False)

Now we have a CSV file written for both EDM and Non-EDM sample sets. In the file 'Model.ipynb' I process these datasets and use them to classify songs.